In [1]:
import os

os.sys.path.insert(0, 'src/')
import pdb_numpy
from pdb_numpy.analysis import compute_pdockQ

In [2]:
%load_ext line_profiler

In [3]:
file = "src/pdb_numpy/tests/input/1rxz_11587_unrelaxed_rank_001_alphafold2_multimer_v3_model_4_seed_000.pdb"
test = pdb_numpy.Coor(file)


In [6]:
%timeit test = pdb_numpy.Coor(file)

12.2 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%timeit test = pdb_numpy.Coor(file)

12.1 ms ± 23.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
%lprun -f pdb_numpy.format.pdb.parse pdb_numpy.Coor(file)

Timer unit: 1e-09 s

Total time: 0.0416507 s
File: /home/murail/Documents/Code/pdb_numpy/src/pdb_numpy/format/pdb.py
Function: parse at line 24

Line #      Hits         Time  Per Hit   % Time  Line Contents
    24                                           def parse(pdb_lines, pqr_format=False):
    25                                               """Parse the pdb lines and return atom information's as a dictionary
    26                                           
    27                                               Parameters
    28                                               ----------
    29                                               pdb_lines : list
    30                                                   list of pdb lines
    31                                               pqr_format : bool, optional
    32                                                   if True, parse pqr format, by default False
    33                                           
    34                    

In [10]:
%timeit test.select_atoms("chain A and name CA")

83.7 µs ± 4.15 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
test_2 = test.select_atoms("chain A and name CA")
test_2.chain[:] = 'z'

In [11]:
import numpy as np
np.unique(test.chain)

array(['A', 'B'], dtype='<U2')

In [12]:
test.get_aa_seq()

{'A': 'MIDVIMTGELLKTVTRAIVALVSEARIHFLEKGLHSRAVDPANVAMVIVDIPKDSFEVYNIDEEKTIGVDMDRIFDISKSISTKDLVELIVEDESTLKVKFGSVEYKVALIDPSAIRKEPRIPELELPAKIVMDAGEFKKAIAAADKISDQVIFRSDKEGFRIEAKGDVDSIVFHMTETELIEFNGGEARSMFSVDYLKEFCKVAGSGDLLTIHLGTNYPVRLVFELVGGRAKVEYILAPRIESE',
 'B': 'KSTQATLERWF'}

In [9]:
%timeit test.get_aa_DL_seq()

56.3 ms ± 680 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%timeit test.get_aa_seq()

782 µs ± 4.49 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [15]:
#%lprun -f compute_pdockQ compute_pdockQ(test)

In [16]:
%lprun -f pdb_numpy.Model.simple_select_atoms compute_pdockQ(test)

Timer unit: 1e-09 s

Total time: 0.000542098 s
File: /home/murail/Documents/Code/pdb_numpy/src/pdb_numpy/model.py
Function: simple_select_atoms at line 225

Line #      Hits         Time  Per Hit   % Time  Line Contents
   225                                               def simple_select_atoms(self, column, values, operator="=="):
   226                                                   """Select atoms from the PDB file based on the selection tokens.
   227                                                   Selection tokens are simple selection containing only one
   228                                                   keyword, operator, and values.
   229                                           
   230                                                   The keywords :
   231                                           
   232                                                   - `"resname"`
   233                                                   - `"chain"`
   234                      

# test array = val

In [22]:
import random
N = 10000

test = np.array([random.randint(1, 10) for i in range(N)])
test

array([8, 3, 1, ..., 2, 5, 3])

In [28]:
%timeit test == 5

4.68 µs ± 24.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [24]:
test == 5

array([False, False, False, ..., False,  True, False])

In [29]:
%timeit np.where(test == 5)

13.6 µs ± 121 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [30]:

def hy36decode(width, s):
    "decodes base-10/upper-case base-36/lower-case base-36 hybrid"
    if len(s) == width:
        f = s[0]
        if f == "-" or f == " " or f.isdigit():
            try:
                return int(s)
            except ValueError:
                pass
            if s == " " * width:
                return 0
        elif f in digits_upper_values:
            try:
                return (
                    decode_pure(digits_values=digits_upper_values, s=s)
                    - 10 * 36 ** (width - 1)
                    + 10**width
                )
            except KeyError:
                pass
        elif f in digits_lower_values:
            try:
                return (
                    decode_pure(digits_values=digits_lower_values, s=s)
                    + 16 * 36 ** (width - 1)
                    + 10**width
                )
            except KeyError:
                pass
    raise ValueError("invalid number literal.")


In [36]:
%timeit hy36decode(5,"78965")

347 ns ± 2.95 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [3]:
%load_ext Cython

In [92]:
len(digits_upper_values)

36

In [55]:
%%cython -a
import cython
from libc.string cimport strcpy, strlen

digits_upper = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
digits_lower = digits_upper.lower()
digits_upper_values = dict([pair for pair in zip(digits_upper, range(36))])
digits_lower_values = dict([pair for pair in zip(digits_lower, range(36))])

cdef int convert_low(char c):
    if c >= 48 and c <=57: #'0':48 '9':57
        return c - 48
    else:
        #print(chr(c), c - 87)
        return c - 87 # -97 'a' + 10

cdef int convert_Cap(char c):
    if c >= 48 and c <=57: #'0':48 '9':57
        return c - 48
    elif c >= 65 and c <=90: #'A':65 'Z':90
        #print(chr(c), c - 87)
        return c - 55 # -65 'A' + 10
    raise ValueError("invalid number literal.")

def decode_pure_cython(digits_values, s):
    """ Decodes the string s using the digit, value
    associations for each character.

    Parameters
    

    """
    result = 0
    n = len(digits_values)
    for c in s:
        result *= n
        result += digits_values[c]
    return result

cdef int decode_pure_cython_low(char * s):
    """ Decodes the string s using value associations for each character
    """
    cdef:
        int result = 0, n = 36
        char c
    
    for c in s:
        result *= n
        result += convert_low(c)
    return result


cdef int decode_pure_cython_Cap(char * s):
    "decodes the string s using value associations for each character"
    cdef:
        int result = 0, n = 36
        char c
    
    for c in s:
        result *= n
        result += convert_Cap(c)
    return result

cdef bint isDigit(char c):
    if c >= 48 and c <=57: #'0':48 '9':57
        return True
    else:
        return False

cdef bint isCapLetter(char c):
    if c >= 65 and c <=90: #'A':65 'Z':90
        return True
    else:
        return False

cdef bint isLowLetter(char c):
    if c >= 97 and c <=122: #'a':97 'z':122   
        return True
    else:
        return False
        
cpdef int hy36decode_cython(int width, str s):
    "decodes base-10/upper-case base-36/lower-case base-36 hybrid"

    tmp = s.encode('utf-8')

    cdef:
        char f
        char * number_str
    number_str = tmp

    if strlen(number_str) == width:
        f = number_str[0]
        if f == b"-" or f == b" " or isDigit(f):
            try:
                return int(s)
            except ValueError:
                pass
            if s == " " * width:
                return 0
        elif isCapLetter(f):
            print("Cap")
            try:
                print(decode_pure_cython_Cap(number_str))
                return (int(
                    decode_pure_cython_Cap(number_str)
                    #decode_pure_cython(digits_values=digits_upper_values, s=s)
                    - 10 * 36 ** (width - 1)
                    + 10**width)
                )
            except KeyError:
                pass
        elif isLowLetter(f):
            try:
                return (int(
                    decode_pure_cython_low(s=number_str)
                    + 16 * 36 ** (width - 1)
                    + 10**width)
                )
            except KeyError:
                pass
    raise ValueError("invalid number literal.")


Content of stderr:
/home/murail/.cache/ipython/cython/_cython_magic_d3a9c3793756c7b7d5c6316fdeedf665d4f88d0d.c: In function ‘__pyx_f_54_cython_magic_d3a9c3793756c7b7d5c6316fdeedf665d4f88d0d_hy36decode_cython’:
/home/murail/.cache/ipython/cython/_cython_magic_d3a9c3793756c7b7d5c6316fdeedf665d4f88d0d.c:3452:43: warning: comparison of integer expressions of different signedness: ‘size_t’ {aka ‘long unsigned int’} and ‘int’ [-Wsign-compare]
 3452 |   __pyx_t_3 = (strlen(__pyx_v_number_str) == __pyx_v_width);
      |                                           ^~

In [56]:
hy36decode_cython(5, "ABCD-")

Cap


ValueError: invalid number literal.

In [28]:
decode_pure_cython(digits_upper_values, "A8965")

17181293

In [29]:
decode_pure_cython_low(b"A000Z")

NameError: name 'decode_pure_cython_low' is not defined

In [30]:
hy36decode_cython(5,"A000Z")

100035

In [80]:
chr(65)

'A'

In [88]:
print(ord('a'),ord('z'))

97 122


In [149]:
%timeit hy36decode_cython(5,"a8965")

160 ns ± 0.73 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [150]:
%timeit hy36decode(5,"a8965")

1.22 µs ± 15 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [81]:
digits_upper_values

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 'A': 10,
 'B': 11,
 'C': 12,
 'D': 13,
 'E': 14,
 'F': 15,
 'G': 16,
 'H': 17,
 'I': 18,
 'J': 19,
 'K': 20,
 'L': 21,
 'M': 22,
 'N': 23,
 'O': 24,
 'P': 25,
 'Q': 26,
 'R': 27,
 'S': 28,
 'T': 29,
 'U': 30,
 'V': 31,
 'W': 32,
 'X': 33,
 'Y': 34,
 'Z': 35}

In [116]:
len(digits_lower_values)

36

In [32]:
%%cython -a


digits_upper = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
digits_lower = "0123456789abcdefghijklmnopqrstuvwxyz"

def encode_pure(str digits, int value):
    """ Encodes value using the given digits.
    """
    cdef:
        int n
        list result = []
        int rest

    assert value >= 0

    if value == 0:
        return digits[0]
    
    n = 36
    result = []

    while value != 0:
        rest = value // n
        result.append(digits[value - rest * n])
        value = rest
    
    result.reverse()
    return "".join(result)


def hy36encode(int width, int value):
    """ Encodes value as base-10/upper-case base-36/lower-case
    base-36 hybrid.
    """
    cdef:
        double i = value
        
    
    if i >= 1 - 10 ** (width - 1):

        if i < 10**width:
            return ("%%%dd" % width) % i
        
        i -= 10**width

        if i < 26 * 36 ** (width - 1):
            i += 10 * 36 ** (width - 1)
            return encode_pure(digits_upper, int(i))
        
        i -= 26 * 36 ** (width - 1)
        if i < 26 * 36 ** (width - 1):
            i += 10 * 36 ** (width - 1)
            return encode_pure(digits_lower, int(i))
        
    raise ValueError("value out of range.")


In [33]:
hy36encode(5, 100035)

'A000Z'

In [47]:
hy36decode_cython(5, "ABCD-")

629226

In [45]:
chr(90)

'Z'

In [46]:
ord('-')

45

In [20]:
digits_upper.lower()

'0123456789abcdefghijklmnopqrstuvwxyz'